In [ ]:
%maven ai.djl:api:0.8.0
%maven ai.djl.pytorch:pytorch-engine:0.8.0
%maven ai.djl.pytorch:pytorch-model-zoo:0.8.0
%maven org.slf4j:slf4j-api:1.7.26
%maven org.slf4j:slf4j-simple:1.7.26
%maven net.java.dev.jna:jna:5.3.0
        
// See https://github.com/awslabs/djl/blob/master/pytorch/pytorch-engine/README.md
// for more PyTorch library selection options
%maven ai.djl.pytorch:pytorch-native-auto:1.6.0

In [ ]:
import ai.djl.*;
import ai.djl.translate.*;
import ai.djl.inference.*;
import ai.djl.ndarray.*;
import ai.djl.modality.*;
import ai.djl.repository.zoo.*;
import ai.djl.repository.zoo.*;
import ai.djl.util.Utils;
import ai.djl.repository.zoo.*;
import ai.djl.modality.cv.*;
import ai.djl.ndarray.types.*;
import ai.djl.modality.cv.util.*;

import java.io.IOException;
import java.nio.file.Path;
import java.nio.file.Paths;
import java.util.List;

1. 请完成 Criteria 代码读取项目内的`tracedd_resnet18.pt`的模型

In [ ]:
public Classifications predict() throws IOException, ModelException, TranslateException {
        Path imageFile = Paths.get("image.jpg");
        Image img = ImageFactory.getInstance().fromFile(imageFile);
    
        // TODO implement model loading using criteria
        Criteria<Image, Classifications> criteria = ...

        try (ZooModel<Image, Classifications> model = ModelZoo.loadModel(criteria)) {
            try (Predictor<Image, Classifications> predictor = model.newPredictor()) {
                return predictor.predict(img);
            }
        }
    }

In [ ]:
class MyTranslator implements Translator<Image, Classifications> {
        private List<String> classes;
        private SynsetLoader synsetLoader = new SynsetLoader("synset.txt");

        static final class SynsetLoader {
            private String synsetFileName;

            public SynsetLoader(String synsetFileName) {
                this.synsetFileName = synsetFileName;
            }

            public List<String> load(Model model) throws IOException {
                return model.getArtifact(synsetFileName, Utils::readLines);
            }
        }

        @Override
        public void prepare(NDManager manager, Model model) throws IOException {
            if (classes == null) {
                classes = synsetLoader.load(model);
            }
        }

        @Override
        public Batchifier getBatchifier() {
            return Batchifier.STACK;
        }

        @Override
        public Classifications processOutput(TranslatorContext ctx, NDList list) {
            NDArray probabilitiesNd = list.singletonOrThrow();
            // TODO Implement softmax here
            return new Classifications(classes, probabilitiesNd);
        }

        @Override
        public NDList processInput(TranslatorContext ctx, Image input) {
            NDArray array = input.toNDArray(ctx.getNDManager(), Image.Flag.COLOR);
            array = NDImageUtils.resize(array, 256, 256);
            array = NDImageUtils.centerCrop(array, 224, 224);

            // TODO Implement toTensor() here
            // remove the reshape here
            array = array.reshape(new Shape(3, 224, 224));

            array = NDImageUtils.normalize(array, new float[] {0.485f, 0.456f, 0.406f}, new float[] {0.229f, 0.224f, 0.225f});
            return new NDList(array);
        }
    }

In [ ]:
predict()

2. 代码里会读去一张萨摩耶的图片，但是目前的分类结果并不是萨摩耶，因为数据前处理的代码有问题，请把第77行的`reshape` 移除并且实现 toTensor()

注意不能直接调用 NDImageUtils.toTensor()
tips:
* toTensor() 方法做了三件事
    1. 把 NDArray 里面每个元素从0到255归一化到0到1
    2. 把 NDArray shape 从 (Height, Width, Channel) 转成 (Channel, Height, Width)
    3. 把数据形态从 uint8 转成 float32
    原来的实现方法是用了`reshape`但底下的数据没有发生任何改变，我们需要用一个会将底下数据排列方式改变的算子．

3. 完成任务 2 之后会发现分类结果正确显示出 Samoyed 但标签后面概率的值是错的，因为在`processOutput()`里面没有加上 softmax()，请实现 softmax算子


注意不能直接调用 array.softmax()